Dans ce notebook, au lieu de prédire si une personne veut revoir une autre personne, nous essayons de predire si en fonction des reponses des deux individus concerné, un match vase realiser (c'est a dire que les deux personnes veulent se revoir)

In [264]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

In [265]:
table_id=pd.read_csv("Speed_Dating_Data.csv", encoding = "ISO-8859-1")

In [266]:
list_qual=["attr","sinc","intel","fun","amb","shar","dec",'iid','gender','age','field_cd','race','imprace','imprelig','date','go_out',
   'sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga',
   'attr1_1', 'sinc1_1', 'intel1_1','fun1_1', 'amb1_1', 'shar1_1','attr3_1', 'sinc3_1', 'intel3_1','fun3_1', 'amb3_1',"pid"]
table_id=table_id[(table_id.wave<6)|(table_id.wave>9)]
table_id=table_id[list_qual]
table_id["somme"]=table_id.attr1_1+ table_id.sinc1_1+table_id.intel1_1+table_id.amb1_1+table_id.shar1_1 +table_id.fun1_1
table_id=table_id[(table_id.somme<110)&(table_id.somme>95)].drop("somme",1)
table_id=table_id.dropna()
table_id.pid=table_id.pid.astype(int)
table_id["match"]=table_id.iid.astype(str)+"-" + table_id.pid.astype(str)

In [267]:
liste_field = ['Law', 'Math', 'Social Science', 'Medical', 'Engineering', 'English', 'History', 'Finance',
         'Education','Physics', 'Social Work', 'Undergard','Politics','Film','Arts','Languages','Architecture', 'Other']
dico_field = {i:liste_field[i-1] for i in range(1,19)}
table_dummies = pd.get_dummies(table_id['field_cd'])
table_id = pd.concat([table_id, table_dummies], axis=1)
table_id.drop('field_cd', axis=1, inplace=True)
table_id.rename(columns=dico_field,inplace=True)

In [268]:
table_id2=table_id.copy()
dic={i:i+"_partner" for i in table_id.columns}
table_id2 = table_id2.rename(columns=dic)
table_id2=table_id2.rename(columns={"match_partner":'match'})
l=table_id.match.str.split("-").values
m=[]
for i in l:
    m.append(i[1]+"-"+i[0])
table_id2.match= np.array(m)

In [269]:
table_id=pd.merge(table_id2,table_id,how="outer",left_on='match',right_on='match').dropna()

La variable "success" vaut 1 si il y a un match, zero sinon.

In [270]:
table_id["success"]=table_id.dec[table_id.dec_partner==1]
table_id.success=np.nan_to_num(table_id.success)


In [279]:
def split(frac,df):
    train=df.sample(frac=frac,random_state=200)
    test=df.drop(train.index)
    return(train,test)
def XY(frac,df):
    train,test=split(frac,df)
    Ytrain=train.success.values
    Ytest=test.success.values
    Xtrain=train.drop(["dec","dec_partner","iid","iid_partner","pid","pid_partner","match","success"],1).values
    Xtest=test.drop(["dec","dec_partner","iid","iid_partner","pid","pid_partner","match","success"],1).values
    return(Xtrain,Xtest,Ytrain,Ytest)

In [280]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_curve, auc, confusion_matrix

In [281]:
Xtrain,Xtest,Ytrain,Ytest=XY(0.8,table_id)

In [282]:
clf=RandomForestClassifier()
clf.fit(Xtrain,Ytrain)
Ypred=clf.predict(Xtest)
print(f1_score(Ytest,Ypred),clf.score(Xtrain,Ytrain))

0.172972972973 0.983717012914


In [283]:
confusion_matrix(Ytest,Ypred)

array([[721,  20],
       [133,  16]])